![logo](https://github.com/jeanmatheuss/SB-2023-python/blob/main/imgs/logo.jpg?raw=true)

---
## **Projeto 1** | Explorando IA Generativa em Pipeline de ETL com Python
[Jean Matheus](https://www.linkedin.com/in/jean-matheus-sm/)

---


# Personalização de Mensagens de Marketing com IA Generativa

## Descrição
Este projeto foi desenvolvido como parte do Santander Bootcamp 2023 - Ciência de Dados com Python, com o obejtivo de envovler os clientes de maneira mais personalizada usando o poder da Inteligência Artificial Generativa.

## Contexto
Como cientista de dados no Santander, nossa tarefa era melhorar o envolvimento dos clientes por meio de mensagens de marketing altamente personalizadas.    
O objetivo é utilizar  a Inteligência Artificial Generativa para criar mensagens que ressoem com cada cliente individualmente.

Aqui estão as estapas que seguiremos:
1. Extração de Dados
2. Obtenção de Dados  dos Clientes
3. Transformação de Dados
4. Geração de Mensagens Personalizadas
5. Atualização de Dados dos Clientes

### Pacotes e Bibliotecas

In [19]:
import json
import pandas as pd
import seaborn as sns
import numpy as np
import requests

In [ ]:
# Criando DF a partir do arquivo SDW2023-1 baixado diretamente da API

df = pd.read_json('SDW2023-1.json')

In [2]:
# URL da API para a consulta dos clientes

url = 'https://sdw-2023-prd.up.railway.app'

In [11]:
# Criando DF diretamente da API

response = requests.get(f'{url}/users')
clients = response.json() if response.status_code == 200 else None
clients_df = pd.DataFrame(clients)

In [4]:
clients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2060 entries, 0 to 2059
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2060 non-null   int64 
 1   name      2060 non-null   object
 2   account   2060 non-null   object
 3   card      2060 non-null   object
 4   features  2060 non-null   object
 5   news      2060 non-null   object
dtypes: int64(1), object(5)
memory usage: 96.7+ KB


> Como usaremos esse projeto apenas para demonstração, usar o *clientes_df* ficaria inviável pois são quase 1400 clientes fictícios e como a API do ChatGPT que irá gerar a mensagem não é gratuíta, vamos usar apenas os 5 prmeiros clientes.

In [12]:
clients_df = clients_df.sort_values(by='id', ascending=True)

In [14]:
clients_df = clients_df.head()
clients_df

,id,name,account,card,features,news
6,1,Devweekerson,"{'id': 1, 'number': '01.097954-4', 'agency': '...","{'id': 1, 'number': 'xxxx xxxx xxxx 1111', 'li...","[{'id': 1, 'icon': 'https://digitalinnovationo...","[{'id': 1, 'icon': 'https://digitalinnovationo..."
10,13,anguile,"{'id': 20, 'number': '00010-8', 'agency': '200...","{'id': 13, 'number': '**** **** **** 0112', 'l...","[{'id': 10, 'icon': 'string', 'description': '...","[{'id': 24, 'icon': 'string', 'description': '..."
11,14,Maria,"{'id': 21, 'number': '00110-2', 'agency': '200...","{'id': 14, 'number': '**** **** **** 0322', 'l...","[{'id': 11, 'icon': 'string', 'description': '...","[{'id': 25, 'icon': 'string', 'description': '..."
12,15,Pyterson,"{'id': 22, 'number': '0001-1', 'agency': '0001...","{'id': 15, 'number': '1234 1234 1234 1111', 'l...",[],"[{'id': 51, 'icon': 'https://digitalinnovation..."
118,17,Arthur,"{'id': 905, 'number': '**** **** **45', 'agenc...","{'id': 836, 'number': '**** **** **67', 'limit...","[{'id': 305, 'icon': 'string', 'description': ...","[{'id': 2086, 'icon': 'string', 'description':..."


In [15]:
lista_id = clients_df['id'].to_list()
lista_id

[1, 13, 14, 15, 17]

In [14]:
# Flatten da coluna account
clients_account = pd.json_normalize(clients_df['account'])
clients_account

,id,number,agency,balance,limit
0,1,01.097954-4,2030,624.12,1000.0
1,20,00010-8,2003,500.00,1000.0
2,21,00110-2,2003,500.00,1000.0
3,22,0001-1,0001,0.00,500.0
4,905,**** **** **45,Banku,0.00,0.0


In [15]:
# Flatten da coluna card
clients_card = pd.json_normalize(clients_df['card'])
clients_card

,id,number,limit
0,1,xxxx xxxx xxxx 1111,2000.0
1,13,**** **** **** 0112,500.0
2,14,**** **** **** 0322,500.0
3,15,1234 1234 1234 1111,1000.0
4,836,**** **** **67,0.0


In [35]:
teste = clients[0]
teste['name']

'leonardo fogaca'

In [41]:
# API ChatGpt
import os
from getpass import getpass

api_key = getpass()


In [ ]:
import openai

openai.api_key = api_key

def gerenate_ai_news(user):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Você é um especialista em marketing bancário."
            },
            {
                "role": "user",
                "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo 100 caracteres)"
            }
        ]
    )
    return completion.choices[0].message.content.strip('\"')


for user in clients[0:4]:
    news = gerenate_ai_news(user)
    print(news)